### Importieren der Bibliotheken

In [1]:
import pypsa
import pandas as pd
import numpy as np

### Importieren der Datensätze

In [2]:
#Datensätze hinterlegen
data_pv = 1
data_wind = 1
data_hydrogen = 100

### Definieren der Variablen

In [3]:
#marginal cost
marginal_cost_grid = 1 #€/kWh
marginal_cost_hydrogen = 1 #€/kWh
marginal_cost_wind = 0.08 #€/kWh
marginal_cost_pv = 0.065 #€/kWh

marginal_cost_electrolysis = 0.1 #€/kWh
marginal_cost_liquefaction = 0.1 #€/kWh

marginal_cost_shipping = 1 #€/kWh
marginal_cost_trailer = 1 #€/kWh
marginal_cost_pipeline = 1 #€/kWh

#capital_cost 
capital_cost_electrolysis = 10 #€/kW annual
capital_cost_liquefaction = 1 #€/kWh annual 
capital_cost_hydrogen_store = 1 #€/kWh annual 

#settings for efficiency of components
efficiency_electrolysis = 1
efficiency_liquefaction = 1

### Aufbau des Netzwerkes

In [4]:
network = pypsa.Network()
network.set_snapshots(range(8760))

In [5]:
#Buses
network.add('Bus', name = 'electricity')
network.add('Bus', name = 'G_hydrogen')
network.add('Bus', name = 'L_hydrogen')
network.add('Bus', name = 'destination')

#Generators
network.add('Generator', name = 'grid', bus = 'electricity', 
            p_nom_extendable = True, 
            marginal_cost = marginal_cost_grid)
network.add('Generator', name = 'hydrogen_supply', bus = 'G_hydrogen', 
            p_nom_extendable = True, 
            marginal_cost = marginal_cost_hydrogen)

#network.add('Generator', name = 'pv', bus = 'electricity', 
#            p_nom_extendable = True, p_max_pu = data_pv,
#            marginal_cost = marginal_cost_pv)
#network.add('Generator', name = 'wind_turbine', bus = 'electricity', 
#            p_nom_extendable = True, p_max_pu = data_wind,
#            marginal_cost = marginal_cost_wind)

#Load
network.add('Load', name = 'hydrogen_load', bus = 'L_hydrogen', p_set = data_hydrogen)

#Links
network.add('Link', name = 'liquefaction', bus0 = 'G_hydrogen', bus1 = 'L_hydrogen', 
            p_nom_extendable = True, efficiency = efficiency_liquefaction, 
            capital_cost = capital_cost_liquefaction,
            marginal_cost = marginal_cost_liquefaction)


#TODO: Transportwege hinzufügen
#network.add('Link', name = 'shipping', bus0 = 'hydrogen', bus1 = 'destination',
#            marginal_cost = marginal_cost_shipping)
#network.add('Link', name = 'trailer', bus0 = 'hydrogen', bus1 = 'destination',
#            marginal_cost = marginal_cost_trailer)
#network.add('Link', name = 'pipeline', bus0 = 'hydrogen', bus1 = 'destination',
#            marginal_cost = marginal_cost_pipeline)

#Stores
network.add('Store', name = 'hydrogen_store', e_cyclic = True, 
            bus = 'L_hydrogen', e_nom_extendable = True, capital_cost = capital_cost_hydrogen_store)

### Optimierung des Modells

In [6]:
network.lopf(pyomo = False, solver_name = 'gurobi')

INFO:pypsa.linopf:Prepare linear problem


ValueError: Empty LHS in nodal balance constraint.

### Auswertung der Ergebnisse